In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt

from sklearn import ensemble
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
import operator
import collections
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle

/home/ephlius/.virtualenvs/3/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# #############################################################################
# Load data

data = pd.read_csv("HM_cohort_wth_LabTest.csv", index_col=0)
data['urea']=data['bun']
data.loc[:,'leukemia'] = pd.Series(1, index=data.index[data['d_type']=='leukemia'])
data.loc[:,'lymphoma'] = pd.Series(1, index=data.index[data['d_type']=='lymphoma'])
data.loc[:,'myeloma'] = pd.Series(1, index=data.index[data['d_type']=='myeloma'])
data.loc[:,'myelodysplastic_syndrome'] = pd.Series(1, index=data.index[data['d_type']=='myelodysplastic_syndrome'])
data[['p_blood_culture_1', 'rbc_1', 'platelet_1', 'vasopressor_1', 
      'ventilation_1','leukemia','lymphoma','myelodysplastic_syndrome',
      'myeloma']] = data[['p_blood_culture_1', 'rbc_1', 'platelet_1', 'vasopressor_1', 'ventilation_1',
                          'leukemia','lymphoma','myelodysplastic_syndrome','myeloma']].fillna(0)    

df = pd.DataFrame(data, columns =['age', 'sofa','wbc','hemoglobin','platelet','sodium','potassium','creatinine','urea'
                                  'bilirubin',
                                  'ast',
                                  'alt',
                                  'abg_ph', 'abg_pco2', 
                                  #'abg_po2', 
                                  'abg_bicarbonate','abg_baseexcess', 
                                  #'abg_spo2', 
                                  'abg_lactate',
                                  'gender',
                                  'p_blood_culture_1',
                                  'rbc_1', 'platelet_1','ventilation_1','vasopressor_1', 'rrt',
                                  'hypertension', 'congestive_heart_failure', 'cardiac_arrhythmias', 'chronic_pulmonary',
                                  'depression', 'diabetes_uncomplicated', 'hypothyroidism',
                                  'renal_failure', 'rheumatoid_arthritis', 'liver_disease','peptic_ulcer',
                                  #'leukemia','lymphoma','myeloma','myelodysplastic_syndrome',
           
                                  'mort_icu',
                                  'mort_hosp'
                                  
                                ])


df['gender'] = df['gender'].str.replace('M','1')
df['gender'] = df['gender'].str.replace('F','0')

#concert the unit for creatinine from mg/dl to umol/l
df['creatinine']=df['creatinine']*88.42
#convert the unit for bilirubin from mg/dl to umol/l
df['bilirubin']=df['bilirubin']*17.1
#convert the unit for BUN (urea) from mg/dL to mmol/L
df['urea']=df['urea']** 0.3571

#df=df.dropna(axis = 0, how='any')
#df=df.fillna(df.median())


mimic_X = pd.DataFrame(df,columns =[
                              'age', 
                              'sofa','wbc','hemoglobin','platelet','sodium','potassium','creatinine','urea'
                              'bilirubin',
                              'ast',
                              'alt',
                              'abg_ph', 'abg_pco2', 
                              #'abg_po2', 
                              'abg_bicarbonate','abg_baseexcess', 
                              #'abg_spo2', 
                              'abg_lactate',
                              'gender',
                              'p_blood_culture_1', 
                              'rbc_1', 
                              'platelet_1','ventilation_1','vasopressor_1', 'rrt',
                              #'hypertension', 'congestive_heart_failure', 'cardiac_arrhythmias', 'chronic_pulmonary',
                              #'depression', 'diabetes_uncomplicated', 'hypothyroidism',
                              #'renal_failure', 'rheumatoid_arthritis', 'liver_disease','peptic_ulcer',
                              #'leukemia','lymphoma','myeloma','myelodysplastic_syndrome'
                                 ])

mimic_y = pd.DataFrame(df,columns =['mort_icu'])
#mimic_y = pd.DataFrame(df,columns =['mort_hosp'])

KeyError: 'bilirubin'

In [6]:
NUHdata = pd.read_csv("NUH_HM data.csv", index_col=0)
NUHdata[['p_blood_culture_1', 'rbc_1', 'platelet_1', 'vasopressor_1', 
      'ventilation_1','leukaemia','lymphoma','myelodysplastic_syndrome',
      'myeloma']] = NUHdata[['p_blood_culture_1', 'rbc_1', 'platelet_1', 'vasopressor_1', 'ventilation_1',
                          'leukemia','lymphoma','myelodysplastic_syndrome','myeloma']].fillna(0)

NUHdf = pd.DataFrame(NUHdata, columns =[
                                  'age', 
    
                                  'sofa','wbc','hemoglobin','platelet','sodium','potassium','creatinine','urea',
                                  'bilirubin',
                                  'ast', 
                                  'alt',
                                  'abg_ph', 'abg_pco2', 
                                  #'abg_po2', 
                                  'abg_bicarbonate','abg_baseexcess',
                                  #'abg_spo2', 
                                  'abg_lactate',
                                  'gender',
                                  'p_blood_culture_1',
                                  'rbc_1', 'platelet_1','ventilation_1','vasopressor_1', 'rrt',
                                  #'hypertension', 'congestive_heart_failure', 'cardiac_arrhythmias', 'chronic_pulmonary',
                                  #'depression', 'diabetes_uncomplicated', 'hypothyroidism',
                                  #'renal_failure', 'rheumatoid_arthritis', 'liver_disease','peptic_ulcer',
                                  #'leukemia','lymphoma','myeloma','myelodysplastic_syndrome',
                                  'mort_icu',
                                  'mort_hosp'
                                ])



NUHdf['gender'] = NUHdf['gender'].str.replace('Male','1')
NUHdf['gender'] = NUHdf['gender'].str.replace('Female','0')
#NUHdf=NUHdf.fillna(NUHdf.median())
#NUHdf=NUHdf.dropna(axis = 0, how='any')


NUH_X = pd.DataFrame(NUHdf,columns =[
                              'age', 
                              'sofa','wbc','hemoglobin','platelet','sodium','potassium','creatinine','urea',
                              'bilirubin',
                              'ast', 
                              'alt',
                              'abg_ph', 'abg_pco2', 
                              #'abg_po2', 
                              'abg_bicarbonate','abg_baseexcess', 
                              #'abg_spo2', 
                              'abg_lactate',
                              'gender',
                              'p_blood_culture_1',
                              'rbc_1', 
                              'platelet_1','ventilation_1','vasopressor_1', 'rrt',
                              'hypertension', 'congestive_heart_failure', 'cardiac_arrhythmias', 'chronic_pulmonary',
                              'depression', 'diabetes_uncomplicated', 'hypothyroidism',
                              'renal_failure', 'rheumatoid_arthritis', 'liver_disease','peptic_ulcer',
                              #'leukemia','lymphoma','myeloma','myelodysplastic_syndrome'
                                 ])

NUH_y = pd.DataFrame(NUHdf,columns =['mort_icu'])
#NUH_y = pd.DataFrame(NUHdf,columns =['mort_hosp'])

In [7]:
NUH_X = np.array(NUH_X)
NUH_y = np.ravel(NUH_y)
n_samples, n_features = NUH_X.shape

In [13]:

score =[]
# #############################################################################
# Fit classification model
# use XGboost

params = {'objective':'binary:logistic', 
          
          
          'gamma':0.1,
          'learning_rate':0.1,
          'min_child_weight':3, 'max_depth':5,
          'n_estimators':140,'nthread':4,
          'reg_lambda':1,'reg_alpha':0.001,
          'scale_pos_weight':1, 'subsample':0.7,'colsample_bytree':0.8,'scale_pos_weight':1
          }
#kf = KFold(n_splits=10, shuffle=True, random_state=159)
kf = KFold(n_splits=10)
for train_index, test_index in kf.split(NUH_X):
    X_train, X_test = NUH_X[train_index], NUH_X[test_index]
    y_train, y_test = NUH_y[train_index], NUH_y[test_index]
    classifier = xgb.XGBClassifier(**params)
    classifier.fit(X_train, y_train)
    predictions = classifier.predict(X_test)
    fpr, tpr, _ = roc_curve(y_test, predictions)
    roc_auc = auc(fpr, tpr)
    score.append(roc_auc.ravel().copy())

score=np.array(score)
print("10-fold Cross Validation AUC: %0.3f (+/- %0.3f)" % (score.mean(), score.std()))
print("10-fold Cross Validation AUC: %0.3f-%0.3f" % (score.mean()-score.std(), score.mean()+score.std()))
# split data into train and test sets
test_size = 0.3
X_train_final, X_test_final, y_train_final, y_test_final = train_test_split(NUH_X, NUH_y, 
                                                                            test_size=test_size, 
                                                                            random_state=84,
                                                                            shuffle=True)

# fit model no training data
model = xgb.XGBClassifier(**params)
model.fit(X_train_final, y_train_final)
predictions = model.predict(X_test_final)


#ROC curve
fpr, tpr, _ = roc_curve(y_test_final, predictions)
roc_auc = auc(fpr, tpr)
print("Final Model AUC: %0.3f" %roc_auc)
#done with NUH local data

10-fold Cross Validation AUC: 0.740 (+/- 0.093)
10-fold Cross Validation AUC: 0.647-0.833
Final Model AUC: 0.792


In [14]:

score =[]
# #############################################################################
# Fit classification model
# use XGboost

params = {'objective':'binary:logistic', 
          
          
          'gamma':0.1,
          'learning_rate':0.1,
          'min_child_weight':3, 'max_depth':5,
          'n_estimators':140,'nthread':4,
          'reg_lambda':1,'reg_alpha':0.001,
          'scale_pos_weight':1, 'subsample':0.7,'colsample_bytree':0.8,'scale_pos_weight':1
          }
#kf = KFold(n_splits=10, shuffle=True, random_state=159)
kf = KFold(n_splits=10)
for train_index, test_index in kf.split(NUH_X):
    X_train, X_test = NUH_X[train_index], NUH_X[test_index]
    y_train, y_test = NUH_y[train_index], NUH_y[test_index]
    classifier = xgb.XGBClassifier(**params)
    classifier.fit(X_train, y_train)
    predictions = classifier.predict(X_test)
    fpr, tpr, _ = roc_curve(y_test, predictions)
    roc_auc = auc(fpr, tpr)
    score.append(roc_auc.ravel().copy())

score=np.array(score)
print("10-fold Cross Validation AUC: %0.3f (+/- %0.3f)" % (score.mean(), score.std()))
print("10-fold Cross Validation AUC: %0.3f-%0.3f" % (score.mean()-score.std(), score.mean()+score.std()))
# split data into train and test sets
test_size = 0.3
X_train_final, X_test_final, y_train_final, y_test_final = train_test_split(NUH_X, NUH_y, 
                                                                            test_size=test_size, 
                                                                            random_state=84,
                                                                            shuffle=True)

# fit model no training data
model = xgb.XGBClassifier(**params)
model.fit(X_train_final, y_train_final)
predictions = model.predict(X_test_final)


#ROC curve
fpr, tpr, _ = roc_curve(y_test_final, predictions)
roc_auc = auc(fpr, tpr)
print("Final Model AUC: %0.3f" %roc_auc)
#done with NUH local data

10-fold Cross Validation AUC: 0.740 (+/- 0.093)
10-fold Cross Validation AUC: 0.647-0.833
Final Model AUC: 0.792


In [ ]:
# extract feature importance
def get_xgb_imp(xgb, feat_names):
    from numpy import array
    imp_vals = xgb.booster().get_fscore()
    imp_dict = {feat_names[i]:float(imp_vals.get('f'+str(i),0.)) for i in range(len(feat_names))}
    total = array(imp_dict.values()).sum()
    return {k:v for k,v in imp_dict.items()}

features_names = ['age', 
                  'sofa','wbc','hemoglobin','platelet','sodium','potassium','creatinine',
                  'bilirubin',
                  'ast', 
                  'alt',
                  'abg_ph', 'abg_pco2', 
                  #'abg_po2', 
                  'abg_bicarbonate','abg_baseexcess',
                  #'abg_spo2', 
                  'abg_lactate',
                  'gender',
                  'p_blood_culture_1',
                  'rbc_1',
                  'platelet_1','ventilation_1','vasopressor_1', 'rrt']
d=get_xgb_imp(model,features_names)
sorted_d= collections.OrderedDict(sorted(d.items(), key=operator.itemgetter(1)))
#plot features importance
plt.figure(figsize=(10, 6))
plt.barh(range(len(sorted_d)),sorted_d.values(),align='center')
plt.yticks(range(len(sorted_d)),list(sorted_d.keys()))
plt.show()

In [46]:
auc_score=[]



for i in range(100,200):
    params = {'objective':'binary:logistic', 
          
          
          'gamma':0,
          'learning_rate':0.1,
          'min_child_weight':3, 'max_depth':5,
          'n_estimators':140,'nthread':4,
          'reg_lambda':1,'reg_alpha':0.001,
          'scale_pos_weight':1, 'subsample':0.7,'colsample_bytree':0.8,'scale_pos_weight':1
          }  
    score=[]
    score1=[]
    kf = KFold(n_splits=10, shuffle=True)
    for train_index, test_index in kf.split(NUH_X):
        X_train, X_test = NUH_X[train_index], NUH_X[test_index]
        y_train, y_test = NUH_y[train_index], NUH_y[test_index]
        classifier = xgb.XGBClassifier(**params)
        classifier.fit(X_train, y_train)
        predictions = classifier.predict(X_test)
        fpr, tpr, _ = roc_curve(y_test, predictions)
        score.append(auc(fpr, tpr).ravel().copy())
    
    score1=np.array(score)
    auc_score.append(score1.mean())
    
        





np.mean(auc_score)
#3,8 0.67342994176909721
#3,5 0.63828652461831048
#subsample, colsample_bytree 0.7,0.8 0.65701409198929317
#'reg_alpha':0.001,0.65842684711553956
#'n_estimators':160 0.68322942123487618

0.71010641847496059

In [47]:
np.std(auc_score)

0.018174624480559322

In [36]:
auc_score=[]
#X0, X1, y0, y1 = train_test_split(mimic_X, mimic_y, test_size=0.402)


for i in range(1,100):
    score=[]
    params = {'objective':'binary:logistic', 
          
          
          'gamma':0,
          'learning_rate':0.1,
          'min_child_weight':3, 'max_depth':7,
          'n_estimators':140,'nthread':4,
          'reg_lambda':1,'reg_alpha':0.001,
          'scale_pos_weight':1, 'subsample':0.8,'colsample_bytree':0.8,'scale_pos_weight':1
          } 
    
    X_train_final, X_test_final, y_train_final, y_test_final = train_test_split(NUH_X, NUH_y, 
                                                                                test_size=0.3
                                                                                ,shuffle=True
                                                                                )
    # fit model no training data
    model = xgb.XGBClassifier(**params)
    model.fit(X_train_final, y_train_final)
    predictions = model.predict(X_test_final)
    #ROC curve
    fpr, tpr, _ = roc_curve(y_test_final, predictions)
    score.append(auc(fpr, tpr).ravel().copy())
    score=np.array(score)
    auc_score.append(score.mean())
        





np.mean(auc_score)

0.69942611574160529

In [34]:
np.std(auc_score)

0.04242920702332479